<a href="https://colab.research.google.com/github/kerupinen/Compdb_final_repo/blob/main/Final_Project_Rupinen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing and Connecting to Database

In [5]:
pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 8.2 MB/s eta 0:00:00


In [6]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient(host='3.230.203.12',  # host is the hostname for the database
                                port=27018,  # port is the port number that mongo is running on
                                username='compdb_student',  # username for the db
                                password='compdbpass',  # password for the db
                                authSource='admin')  # Since our user only exists for the emse6992 db, we need to specify this

# Setup collections and connections
db = client.twitter
stats_coll = db.twitter_statuses
retweets_coll = db.twitter_retweets
favs_coll = db.twitter_favorites
lists_coll = db.twitter_lists_updated
friends_coll = db.twitter_friends_updated


In [7]:
import psycopg2
import psycopg2.extras
import pandas as pd

# Connect to the database
connection = psycopg2.connect(host='3.230.203.12',
                             user='compdb',
                             port=5438,
                             database='twitter',
                             password='compdbs_postgres')
connection.set_session(readonly=True, autocommit=True)

cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

#Work

##Tweets per weekday

In [10]:
x = 1
weekdayCountList=[]
total = 0
ototal = 0
while x < 8:
    query = "SELECT count(distinct(statuses)) FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    weekdayCountList.append([x,res[0]['count']])

    x=x+1


print(weekdayCountList)


[[1, 6618], [2, 6264], [3, 6684], [4, 6647], [5, 6729], [6, 5172], [7, 4717]]


In [11]:
import plotly.express as px
fig = px.bar(weekdayCountList, x=0, y=1, title = "Tweets by weekday",labels={'0':'Day of Week','1':'Number of Tweets'}, range_y=(4000,7000))
fig.show()

##Retweets and Likes per weekday

In [56]:
x = 1
weekDayTotals=[]
total = 0
ototal = 0
while x < 8:
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for i in res:
        total = total+i['favorites_count']

    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for w in res:
        ototal = ototal+w['retweet_count']

    weekDayTotals.append([x,total,ototal])
    x=x+1
    ototal=0
    total=0


print(weekDayTotals)

[[1, 18805170, 8095532], [2, 20967480, 8806093], [3, 22649066, 11023208], [4, 21073340, 11227533], [5, 28259773, 11119761], [6, 20573004, 16762835], [7, 22489203, 8907312]]


In [57]:
retweetday = []
likesday = []
x=1
for i in weekDayTotals:
    retweetday.append([x,i[2]])
    likesday.append([x,i[1]])

    x=x+1

In [61]:
fig = px.bar(retweetday, x=0, y=1, title = "retweets by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,20000000))
fig.show()

In [62]:
fig = px.bar(likesday, x=0, y=1, title = "Likes by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,30000000))
fig.show()

##Retweets per Tweet and Likes per Tweet per weekday

In [63]:
x = 1
avgRetweetpd = []
avgLikespd = []
while x < 8:
    average = (weekDayTotals[x-1][2])/(weekdayCountList[x-1][1])
    avgRetweetpd.append([x,average])

    average = (weekDayTotals[x-1][1])/(weekdayCountList[x-1][1])
    avgLikespd.append([x,average])
    x=x+1

print(avgRetweetpd)
print(avgLikespd)


[[1, 1223.25959504382], [2, 1405.8258301404853], [3, 1649.1932974266906], [4, 1689.112832856928], [5, 1652.513152028533], [6, 3241.074052590874], [7, 1888.3425906296375]]
[[1, 2841.5185856754306], [2, 3347.2988505747126], [3, 3388.549670855775], [4, 3170.3535429517074], [5, 4199.698766532917], [6, 3977.7656612529004], [7, 4767.691965232139]]


Retweet per tweet

In [67]:
fig = px.bar(avgRetweetpd, x=0, y=1, title = "retweet per tweet by weekday",labels={'0':'Day of Week','1':'Retweets per Tweet'}, range_y=(0,4000))
fig.show()

Like Per Tweet

In [69]:
fig = px.bar(avgLikespd, x=0, y=1, title = "like per tweet by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,5000))
fig.show()

##Tweets per Week of Month

In [26]:
tweetsPerWeek = []
x=0
y=7
i=1
while y<36:
    query = "SELECT count(distinct(status_id)) FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    tweetsPerWeek.append([i,res[0]['count']])
    x=y+1
    if(y==21):
        y=y+10
    else:
        y=y+7
    i=i+1

print(tweetsPerWeek)



[[1, 10374], [2, 11578], [3, 9234], [4, 11645]]


In [27]:
fig = px.bar(tweetsPerWeek, x=0, y=1, title = "Tweets by week",labels={'0':'Week of Month','1':'Number of Tweets'}, range_y=(5000,12000))
fig.show()

##Retweets and Likes per week of Month

In [51]:
totalsPerWeek = []
total = 0
ototal = 0
x=0
y=7
num=1
while y<36:
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for counts in res:
        total = total+counts['favorites_count']


    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for retweets in res:
        ototal = ototal+retweets['retweet_count']

    totalsPerWeek.append([num,total,ototal])
    x=y+1
    if(y==21):
        y=y+10
    else:
        y=y+7
    num=num+1
    ototal=0
    total=0

print(totalsPerWeek)

[[1, 36996688, 25150022], [2, 34127560, 16885833], [3, 35106722, 13797065], [4, 48586066, 20109354]]


In [52]:
retweetweek = []
likesweek = []
x=1
for i in totalsPerWeek:
    retweetweek.append([x,i[2]])
    likesweek.append([x,i[1]])

    x=x+1

In [55]:
fig = px.bar(retweetweek, x=0, y=1, title = "retweets by week of month",labels={'0':'Week of Month','1':'Retweet per Tweet'}, range_y=(0,30000000))
fig.show()

In [71]:
fig = px.bar(likesweek, x=0, y=1, title = "Likes by week of month",labels={'0':'Week of Month','1':'Like per Tweet'}, range_y=(0,50000000))
fig.show()

##Retweets per Tweet and Likes per Tweet per Week of Month

In [74]:
print(totalsPerWeek)
print(tweetsPerWeek)

[[1, 36996688, 25150022], [2, 34127560, 16885833], [3, 35106722, 13797065], [4, 48586066, 20109354]]
[[1, 10374], [2, 11578], [3, 9234], [4, 11645]]


In [75]:
x = 1
avgRetweetpw = []
avgLikespw = []
while x < 5:
    average = (totalsPerWeek[x-1][2])/(tweetsPerWeek[x-1][1])
    avgRetweetpw.append([x,average])

    average = (totalsPerWeek[x-1][1])/(tweetsPerWeek[x-1][1])
    avgLikespw.append([x,average])
    x=x+1

print(avgRetweetpw)
print(avgLikespw)

[[1, 2424.3321765953347], [2, 1458.4412679219208], [3, 1494.1590859865714], [4, 1726.8659510519535]]
[[1, 3566.2895700790436], [2, 2947.621350837796], [3, 3801.8975525232836], [4, 4172.268441391155]]


In [76]:
fig = px.bar(avgRetweetpw, x=0, y=1, title = "retweet per tweet by week of month",labels={'0':'week of month','1':'Retweets per Tweet'}, range_y=(0,4000))
fig.show()

In [77]:
fig = px.bar(avgLikespw, x=0, y=1, title = "like per tweet by week of month",labels={'0':'week of month','1':'Like per Tweet'}, range_y=(0,5000))
fig.show()

##Other Exploration